![lsbu](https://storage-prtl-co.imgix.net/endor/organisations/11506/logos/1540893470_rsz_lsbu_crest_simple_white_text_horizon_blackoutline.jpg)

## Introduction
This notebook shows how to configure the flask application, implementing the `Attendance monitoring via SMS` project. 

## Objectives
-  Import python all libraries and packages
-  Set up the paths to the project directories
-  Configure a flask application
-  Configure falsk-mailing
-  Configure flask-login and seesion management 
-  Define endpoints for flask application

## Configuartion

Make sure you have the required environments and packages configured, as mentioned in the `01a_set_up_the_environments
` notebook.

## Import Libraries

In [ ]:
import os
import re
import pathlib
import secrets
import pandas as pd
from typing import Dict, Optional
from flask import Flask, request, render_template, redirect, url_for
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user
from bs4 import BeautifulSoup
import altair as alt
from flask_mail import Mail, Message
from dotenv import load_dotenv
from datetime import date, datetime
from twilio.rest import Client

## Folder Structure
```python
|-- attendance_monioring_via_sms
|   |-- .twilio
|   |-- datasets
|      |-- *.csv
|   |-- docs
|      |-- *.md
|      |-- *.pdf
|   |-- env_vars
|      |-- .env.txt
|   |-- images
|      |-- *.png
|   |-- nbs
|      |-- *.ipynb 
|   |-- static
|      |- *.css
|      |- *.js
|   |-- templates
|      |- *.html
|   |-- app.py
|   |-- conda_env_twilio.yml
|   |-- requirements.txt

```

## Set Paths

In [ ]:
# Set paths
base_dir = pathlib.Path().absolute().parent

# Path to all data directories
data = pathlib.Path(base_dir/'datasets')

# Path to credentials data
credentials = data/'credentials'

# Path to raw data
raw = data/'raw'

# Path to processed data
processed = data/'processed'

# Path to environment variables
env_vars = pathlib.Path(base_dir/'env_vars')

## Load Environment Vaiables
All the sensitive user information are stored in a `.env.txt` file, which is loaded, using `dotenv`.

In [ ]:
# Load dotenv file
load_dotenv(env_vars/'.env.txt')

## Configure Flask App
The flask app has a flask-mail app to send emails, which is configured with an `smtp` based mail testing software/website called, `Mailtrap`. It creats dummy emails, sender and receiver rather than acutally sending the emails but with the same conventions that apply to the real emails.

The flask_login app tracks the login of the users to the application and used for session management and provide the endpoints from unauthorised access by an adversary.

In [ ]:
# Flask app
app = Flask(__name__)
# Configure a secret-key for the flask app
app.config["SECRET_KEY"] = secrets.token_hex(24)

# Configure flask mail app
app.config['MAIL_SERVER'] = 'smtp.mailtrap.io'
app.config['MAIL_PORT'] = 2525
app.config['MAIL_USERNAME'] = os.environ.get('MAIL_USERNAME')
app.config['MAIL_PASSWORD'] = os.environ.get('MAIL_PASSWORD')
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USE_SSL'] = False

# Create login_manager app
login_manager = LoginManager(app)
# Create mail app
mail = Mail(app)

# Create twilio client object
account_sid = os.environ.get("TWILIO_ACCOUNT_SID")
auth_token = os.environ.get("TWILIO_AUTH_TOKEN")
client = Client(account_sid, auth_token)

# Define a User class to store all the users from the .csv input files
users: Dict[str, "User"] = {}

class User(UserMixin):
    def __init__(self, id: str, username: str, email: str, password: str):
        self.id = id
        self.username = username
        self.email = email
        self.password = password

    @staticmethod
    def get(user_id: str) -> Optional["User"]:
        return users.get(user_id)

    def __str__(self) -> str:
        return f"<Id: {self.id}, Username: {self.username}, Email: {self.email}>"

    def __repr__(self) -> str:
        return self.__str__()

data = pd.read_csv(credentials/'login_credentials.csv')
for index in data.index:
    users[str(index)] = User(
        id=index,
        username=data.loc[index, "first_name"],
        email=data.loc[index, "email"],
        password=data.loc[index, "password"],
    )

###
"""
All the endpoints for the application goes here
"""
###

if __name__ == '__main__':
    app.run(debug=True)

## Endpoint for Adding New Student

In [ ]:
# Endpoint for adding new students
@app.route('/add_students', methods=['GET', 'POST'])
@login_required
def add_students():  
    message = None
    df_students = pd.read_csv(credentials/'students.csv')
    if request.method == 'POST':
        id =len(df_students)+1
        student_id = request.form['student_id']
        test = df_students.student_id.astype(str).isin([str(student_id)]).any()
        if test:
            message = 'Student ID exists! Please check the Student ID, or contact the administrator.'
            return render_template('add_students.html', message=message)
        if request.form['first_name'] == '':
            message = 'First Name is required!'
            return render_template('add_students.html', message=message)
        elif request.form['last_name'] == '':
            message = 'Last Name is required!'
            return render_template('add_students.html', message=message)
        elif request.form['student_id'] == '':
            message = 'Student ID is required!'
            return render_template('add_students.html', message=message)
        elif request.form['email'] == '':
                message = 'Email is required!'
                return render_template('add_students.html', message=message)
        elif ((str(request.form['student_id']).isdigit())==False) or (len(request.form['student_id'])!=6):
            message = 'Student ID must be 6 digits!'
            return render_template('add_students.html', message=message)
        elif str(request.form['email']).endswith('@lsbu.ac.uk')==False: 
            message = 'Invalid Email! Please type a valid LSBU email.'
            return render_template('add_students.html', message=message)
        elif df_students.email.isin([str(request.form['email'])]).any(): 
                message = 'Student Exists! Please add a different Student.'
                return render_template('add_students.html', message=message)
        else:
            first_name = request.form['first_name']
            last_name = request.form['last_name']
            student_id = request.form['student_id']
            email = request.form['email']
            
            df = df_students.append({'id': id, 'first_name': first_name, 'last_name': last_name, 'student_id': student_id,
            'email':email}, ignore_index=True)
            df.to_csv(credentials/'students.csv', index=False)
            message = "Thanks for adding a new student!"
            return render_template('add_students.html', message=message)
    return render_template('add_students.html')

## Endpoint for Adding New Tutors

In [ ]:
# Endpoint for adding new tutors
@app.route('/add_tutors', methods=['GET', 'POST'])
@login_required
def add_tutors():  
    message = None
    df_admins = pd.read_csv(credentials/'admins.csv')
    id =len(df_admins)+1
    if request.method == 'POST':
        if request.form['first_name'] == '':
            message = 'First Name is required!'
            return render_template('add_tutors.html', message=message)
        elif request.form['last_name'] == '':
            message = 'Last Name is required!'
            return render_template('add_tutors.html', message=message)
        elif request.form['email'] == '':
            message = 'Email is required!'
            return render_template('add_tutors.html', message=message)
        elif str(request.form['email']).endswith('@lsbu.ac.uk')==False: 
            message = 'Invalid Email! Please type a valid LSBU email.'
            return render_template('add_tutors.html', message=message)
        elif df_admins.email.isin([str(request.form['email'])]).any(): 
            message = 'Tutor Exists! Please add a different tutor.'
            return render_template('add_tutors.html', message=message)
        else:
            try:
                test = df_admins.loc[df_admins.email==current_user.email, 'admin_status'].values[0]=='Yes'
            except:
                message = 'This Account does not have the admin rights to add a new tutor. Please contact the system admin.'
                return render_template('add_tutors.html', message=message)
            if test:
                first_name = request.form['first_name']
                last_name = request.form['last_name']
                email = request.form['email']
                
                df = df_admins.append({'id': id, 'first_name': first_name, 'last_name': last_name, 'email': email, 
                'description': 'Tutor', 'admin_status': 'No'}, ignore_index=True)
                df.to_csv(credentials/'admins.csv', index=False)
                message = "Thanks for adding a new tutor!"
                return render_template('add_tutors.html', message=message)
            else:
                message = 'This Account does not have the admin rights to add a new tutor. Please contact the system admin.'
                return render_template('add_tutors.html', message=message) 
    return render_template('add_tutors.html')

## Endpoint for Adding New Course

In [ ]:
# Endpoint for adding new courses
@app.route('/add_course', methods=['GET', 'POST'])
@login_required
def add_course():  
    message = None
    df_admins = pd.read_csv(credentials/'admins.csv')
    df_courses = pd.read_csv(credentials/'courses.csv')
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    times = [f'{i:02d}:{0:02d}' for i in range(9,19)]
    length = range(1, 4)
    id_courses =len(df_courses)+1
    id_admins = len(df_admins)+1
    if request.method == 'POST':
        if request.form['course_name'] == '':
            message = 'Course Name is required!'
            return render_template('add_new_course.html', message=message, days=days, times=times, length=length)
        elif request.form['tutor_name'] == '':
            message = 'Tutor Name is required!'
            return render_template('add_new_course.html', message=message, days=days, times=times, length=length)
        elif request.form['email'] == '':
            message = 'Email is required!'
            return render_template('add_new_course.html', message=message, days=days, times=times, length=length)
        elif str(request.form['email']).endswith('@lsbu.ac.uk')==False: 
            message = 'Invalid Email! Please type a valid LSBU email.'
            return render_template('add_new_course.html', message=message, days=days, times=times, length=length)
        elif len(str(request.form['tutor_name']).split(' ')) == 1:
            message = 'Tutor Name should at least contain the firstname and surname'
            return render_template('add_new_course.html', message=message, days=days, times=times, length=length)
        elif df_courses.course_name.isin([str(request.form['course_name'])]).any(): 
            message = 'Course Exists! Please add a different Course.'
            return render_template('add_new_course.html', message=message, days=days, times=times, length=length)
        else:
            try:
                test = df_admins.loc[df_admins.email==current_user.email, 'admin_status'].values[0]=='Yes'
            except:
                message = 'This Account does not have the admin rights to add a new course. Please contact the system admin.'
                return render_template('add_new_course.html', message=message, days=days, times=times, length=length)
            if test:
                course_name = request.form['course_name']
                course_code = df_courses.course_code.max()+1
                tutor = request.form['tutor_name']
                email = request.form['email']
                teaching_day = request.form['teaching_day']
                start_time = request.form['start_time']
                duration = request.form['duration']
                end_time = (pd.to_datetime(pd.Series([f'{start_time}'])) + pd.Timedelta(f"{duration} hour")).dt.strftime('%H:%M')[0]
                
                df_courses = df_courses.append({'id': id_courses, 'course_name': course_name, 'course_code': course_code, 'tutor': tutor, 
                'email': email, 'teaching_day': teaching_day, 'start_time': start_time, 'end_time': end_time}, ignore_index=True)
                df_courses.to_csv(credentials/'courses.csv', index=False)
                message = f"Thanks for adding a new course! The course code is: {int(course_code)}."

                if not(df_admins.email.isin([email]).any()):
                    df_admins = df_admins.append({'id': id_admins, 'first_name': str(tutor).split()[0], 'last_name': str(tutor).split()[1],
                    'email': email, 'description': 'Tutor', 'admin_status': 'No'}, ignore_index=True)
                    df_admins.to_csv(credentials/'admins.csv', index=False)
                return render_template('add_new_course.html', message=message, days=days, times=times, length=length)
            else:
                message = 'This Account does not have the admin rights to add a new course. Please contact the system admin.'
                return render_template('add_new_course.html', message=message, days=days, times=times, length=length) 
    return render_template('add_new_course.html', days=days, times=times, length=length)

## Endpoint for Updating a Course Details

In [ ]:
# Endpoint for updating a courses
@app.route('/update_course', methods=['GET', 'POST'])
@login_required
def update_course():  
    message = None
    df_admins = pd.read_csv(credentials/'admins.csv')
    df_courses = pd.read_csv(credentials/'courses.csv')
    courses = df_courses.course_name.dropna().unique()
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    times = [f'{i:02d}:{0:02d}' for i in range(9,19)]
    length = range(1, 4)

    if request.method == 'POST':
        try:
            test = df_admins.loc[df_admins.email==current_user.email, 'admin_status'].values[0]=='Yes'
        except:
            message = 'This Account does not have the admin rights to update a course. Please contact the system admin.'
            return render_template('update_course.html', message=message, courses=courses, days=days, times=times, length=length)
        if test:
            course_name = request.form['comp_select']
            teaching_day = request.form['comp_select1']
            start_time = request.form['comp_select2']
            duration = request.form['comp_select3']
            end_time = (pd.to_datetime(pd.Series([f'{start_time}'])) + pd.Timedelta(f"{duration} hour")).dt.strftime('%H:%M').squeeze()

            index = df_courses.loc[df_courses.course_name==course_name].index.values[0]

            df_courses.loc[index, 'teaching_day'] = teaching_day
            df_courses.loc[index, 'start_time'] = start_time
            df_courses.loc[index, 'end_time'] = end_time
            df_courses.to_csv(credentials/'courses.csv', index=False)

            message = f"Thanks for updating the course '{course_name}.' Please go to the 'Course List' to check the changes." 
            return render_template('update_course.html', message=message, courses=courses, days=days, times=times, length=length)
        else:
            message = 'This Account does not have the admin rights to update a course. Please contact the system admin.'
            return render_template('update_course.html', message=message, courses=courses, days=days, times=times, length=length)
    return render_template('update_course.html', courses=courses, days=days, times=times, length=length)

The other endpoints are discussed in the later notebooks.

### Summary
In this notebook it was demonstrated how to:
-  Import python all libraries and packages
-  Set up the paths to the project directories
-  Configure a flask application
-  Configure falsk-mailing
-  Configure flask-login and seesion management 
-  Define endpoints for flask application

<center><b>Author</b></center>

| Name | Date Created | Last Modified |
|------|--------------|---------------|
|Khaled Ahmed | 30/01/2023 | 30/01/2023|